## Code flow
----------------------------------

1. Load the dataset
2. Basic preprocessing V
3. Training Process

                    1.   Create the mode
                    2.   Forward pass
                    3.   Loss calculation
                    4.   Backprop
                    5.   Parameters update


4. Model evaluation

In [32]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [33]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [34]:
df.shape

(569, 33)

In [35]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [36]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## Train Test split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

## Scaling - so that all the values should come in same level

In [38]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [39]:
X_train

array([[ 1.49196415, -0.28328947,  1.44147686, ...,  1.69798864,
         1.15973245, -0.31725222],
       [-0.319701  ,  0.64255763, -0.41099901, ..., -1.5800033 ,
        -0.97498839, -1.44892779],
       [-0.33387678, -0.25808076, -0.32204409, ..., -0.71745853,
         0.52838258, -0.63160655],
       ...,
       [-0.42743695,  0.30796932, -0.40525998, ..., -0.04316138,
        -1.08873027,  0.08966199],
       [-0.09572361, -0.83100594, -0.06583754, ...,  0.3788436 ,
        -0.49034906,  1.14565967],
       [-1.4883526 , -1.09455152, -1.36285759, ..., -0.99403877,
        -1.04257415,  1.460014  ]])

In [40]:
y_train

,diagnosis
282,M
178,B
402,B
94,M
496,B
...,...
362,B
459,B
111,B
73,M


## Label Encoder - as M and B (above) cant be understandable by the tranformer so we need to convert M and B into 0 and 1

In [41]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder. transform(y_test)

In [42]:
y_train

array([1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,

## Converting the above Numpy array into Pytorch tensors

In [43]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [44]:
X_train_tensor

tensor([[ 1.4920, -0.2833,  1.4415,  ...,  1.6980,  1.1597, -0.3173],
        [-0.3197,  0.6426, -0.4110,  ..., -1.5800, -0.9750, -1.4489],
        [-0.3339, -0.2581, -0.3220,  ..., -0.7175,  0.5284, -0.6316],
        ...,
        [-0.4274,  0.3080, -0.4053,  ..., -0.0432, -1.0887,  0.0897],
        [-0.0957, -0.8310, -0.0658,  ...,  0.3788, -0.4903,  1.1457],
        [-1.4884, -1.0946, -1.3629,  ..., -0.9940, -1.0426,  1.4600]],
       dtype=torch.float64)

In [45]:
y_train_tensor

tensor([1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0,
        1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1,
        0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
        1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
        1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,

## Defining the model ( creating the structure of the model)

In [46]:
class MySimpleNN():

  def __init__(self, X):

    self.weights = torch.rand(X.shape[1], 1, dtype=torch.float64, requires_grad=True)
    self.bias = torch.zeros(1, dtype=torch.float64, requires_grad=True)


  def forward(self, X):  # this means first we will do z=wx+b and then sigmoid(z)
    z = torch.matmul(X, self.weights) + self.bias
    y_pred = torch.sigmoid(z)
    return y_pred


  def loss_function(self, y_pred, y): # as we are using 0 and 1 so we will use binary cross entropy as loss function
    # Clamp predictions to avoid log(0)
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

    # Calculate loss
    loss = -(y_train_tensor * torch.log(y_pred) + (1-y_train_tensor) * torch.log(1-y_pred)).mean()
    return loss

## Important parameters

In [47]:
learning_rate = 0.1
epochs = 25

## Training pipeline

In [48]:
# create model
model = MySimpleNN(X_train_tensor)
model.weights

# define loop
for epoch in range(epochs):

      # forward pass
      y_pred = model.forward(X_train_tensor)

      # loss calculate
      loss = model.loss_function(y_pred, y_train_tensor)
      print(f'Epoch: {epoch+1} Loss: {loss}')

      # backward propogation ( using autograd)
      loss.backward()

      #parameters update
      with torch.no_grad():
        model.weights -= learning_rate * model.weights.grad
        model.bias -= learning_rate * model.bias.grad

      # zero gradients
      model.weights.grad.zero_()
      model.bias.grad.zero_()

      # print loss in each epoch
      print(f'Epoch: {epoch+1} Loss: {loss}')


Epoch: 1 Loss: 3.5050959518263136
Epoch: 1 Loss: 3.5050959518263136
Epoch: 2 Loss: 3.3719571728470417
Epoch: 2 Loss: 3.3719571728470417
Epoch: 3 Loss: 3.2321994939572907
Epoch: 3 Loss: 3.2321994939572907
Epoch: 4 Loss: 3.0855335951129432
Epoch: 4 Loss: 3.0855335951129432
Epoch: 5 Loss: 2.9315653752102278
Epoch: 5 Loss: 2.9315653752102278
Epoch: 6 Loss: 2.768392545063896
Epoch: 6 Loss: 2.768392545063896
Epoch: 7 Loss: 2.6045379044606576
Epoch: 7 Loss: 2.6045379044606576
Epoch: 8 Loss: 2.4375309263114926
Epoch: 8 Loss: 2.4375309263114926
Epoch: 9 Loss: 2.2702542355679998
Epoch: 9 Loss: 2.2702542355679998
Epoch: 10 Loss: 2.101981784069836
Epoch: 10 Loss: 2.101981784069836
Epoch: 11 Loss: 1.931492270361948
Epoch: 11 Loss: 1.931492270361948
Epoch: 12 Loss: 1.7630437709621938
Epoch: 12 Loss: 1.7630437709621938
Epoch: 13 Loss: 1.5942691971452279
Epoch: 13 Loss: 1.5942691971452279
Epoch: 14 Loss: 1.4349508505422466
Epoch: 14 Loss: 1.4349508505422466
Epoch: 15 Loss: 1.2900990248648063
Epoch: 15

In [49]:
model.weights

tensor([[-0.2679],
        [ 0.1500],
        [ 0.1016],
        [ 0.1177],
        [ 0.0910],
        [ 0.1774],
        [ 0.1179],
        [-0.2352],
        [-0.3197],
        [-0.1724],
        [-0.1807],
        [ 0.2917],
        [-0.1295],
        [ 0.1788],
        [ 0.0109],
        [-0.2629],
        [-0.3192],
        [-0.2869],
        [ 0.5585],
        [ 0.4584],
        [-0.1447],
        [-0.0470],
        [ 0.2048],
        [ 0.2572],
        [ 0.3946],
        [ 0.2046],
        [ 0.2568],
        [-0.2319],
        [ 0.0543],
        [-0.0657]], dtype=torch.float64, requires_grad=True)

In [50]:
model.bias

tensor([-0.1331], dtype=torch.float64, requires_grad=True)

## Model Evaluation

In [51]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.5).float()  # means if value is above 0.5 we call it 1 and if less than 0.5 we will call it 0
  accuracy = (y_pred == y_test_tensor). float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.5323176383972168
